Project 2
Michael Bell
Rob Chesser
Adam Durar

In [1]:
import pandas as pd
import numpy as np
import os
import glob
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
#pip install sqlalchemy-utils for the above

Get list of CSV's from each data set

In [2]:
path = './csv/dj1'
all_files = glob.glob(os.path.join(path, "*.csv"))

li = []

for filename in all_files:
    fn, ext = os.path.splitext(os.path.basename(filename))
    df = pd.read_csv(filename, index_col=None, header=0)
    df = df[['Date','Open','Close']]
    df = df.rename(columns={"Date": "date", "Open": "open", "Close": "close"})
    df['symbol'] = fn
    li.append(df)


In [3]:
path = './csv/dj2'
all_files = glob.glob(os.path.join(path, "*.csv"))

li2 = []

for filename in all_files:
    fn, ext = os.path.splitext(os.path.basename(filename))
    df = pd.read_csv(filename, index_col=None, header=0)
    df = df[['date','open','close']]
    df['symbol'] = fn
    li2.append(df)


Flatten DF Lists into DF

In [4]:
djdata2 = pd.concat(li2, axis=0, ignore_index=True)
djdata = pd.concat(li, axis=0, ignore_index=True)

Flatten DF Lists into final DF

In [5]:
djdata3 = [djdata,djdata2]
djdata4 = pd.concat(djdata3, axis=0, ignore_index=True)

Print Row/Col counts

In [6]:
print(djdata.shape)
print(djdata2.shape)
print(djdata4.shape)

(133296, 4)
(36920, 4)
(170216, 4)


Drop duplicates based on symbol & date columns

In [7]:
cols = ['symbol','date']
djdata4.drop_duplicates(subset=cols, keep='first', inplace=True)
print(djdata4.shape)

(145538, 4)


Connect to database or create if it doesn't exist

In [8]:
connection_string = "postgres:postgres@localhost:5432/tbm"
engine = create_engine(f'postgresql://{connection_string}')
if not database_exists(engine.url):  #pip install sqlalchemy-utils
    create_database(engine.url)

Drop table if it exists

In [9]:
engine.execute('DROP TABLE if exists "stock_data"')

Check to be sure the table is dropped

In [10]:
engine.table_names()

[]

Create fresh table

In [11]:
engine.execute('CREATE TABLE "stock_data" ('
'id INTEGER PRIMARY KEY,'
'date date,'
'open float,'
'close float,'
'symbol varchar'
');')

Check if table is created

In [12]:
engine.table_names()

['stock_data']

Set the index to the id field and reset the index count after concat

In [13]:
djdata4.index.name = 'id'
djdata4.reset_index()
djdata4.head()

,date,open,close,symbol
id,,,,
0,1999-12-31,3.604911,3.671875,AAPL
1,2000-01-03,3.745536,3.997768,AAPL
2,2000-01-04,3.866071,3.660714,AAPL
3,2000-01-05,3.705357,3.714286,AAPL
4,2000-01-06,3.790179,3.392857,AAPL


Insert data into Postgres

In [14]:
djdata4.to_sql(name='stock_data', con=engine, if_exists='append', index=True)

Show sample data

In [15]:
engine.execute('SELECT * FROM stock_data LIMIT 5').fetchall()

[(0, datetime.date(1999, 12, 31), 3.604911, 3.671875, 'AAPL'),
 (1, datetime.date(2000, 1, 3), 3.745536, 3.997768, 'AAPL'),
 (2, datetime.date(2000, 1, 4), 3.866071, 3.660714, 'AAPL'),
 (3, datetime.date(2000, 1, 5), 3.705357, 3.714286, 'AAPL'),
 (4, datetime.date(2000, 1, 6), 3.790179, 3.392857, 'AAPL')]

Check row counts

In [16]:
engine.execute('SELECT count(*) FROM stock_data').fetchall()

[(145538,)]